In [1]:
using Pkg
Pkg.activate("..")

using Flux

using CSV
using DataFrames

using Random
Random.seed!(42)

df = CSV.read("data1.csv", DataFrame)
x = Matrix(df[:, 1:2])
x = [x[i, :] for i in 1:size(x, 1)]
y = df[:, 3]

X = hcat(x...)
y_train = Flux.onehotbatch(y, unique(y))
y_train = Flux.unstack(y_train',1)

  Activating project at `C:\Users\Lenovo\Desktop\SP\LaplaceRedux.jl\dev`


100-element Vector{Vector{Bool}}:
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 [1, 0, 0, 0]
 ⋮
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]
 [0, 0, 0, 1]

In [2]:
data = zip(x,y_train)
n_hidden = 3
D = size(X,1)
out_dim = length(unique(y))
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, out_dim)
)  
loss(x, y) = Flux.Losses.logitcrossentropy(nn(x), y)

loss (generic function with 1 method)

In [ ]:
using Flux.Optimise: update!, Adam
using Statistics
opt = Adam()
epochs = 200
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
    for d in data
        gs = gradient(Flux.params(nn)) do
            l = loss(d...)
        end
        update!(opt, Flux.params(nn), gs)
    end
    if epoch % show_every == 0
        println("Epoch " * string(epoch))
        @show avg_loss(data)
    end
end

In [3]:
py_layer1 = [[-2.7541,  0.5183],
         [ 3.5415,  0.1424],
         [ 0.1192, -4.3529]]

py_layer1 = hcat(py_layer1...)
py_layer1 = py_layer1'

py_layer2 = [-0.9672,  1.5726, -2.0302]

py_layer3 = [[-5.3452,  2.9503, -8.0299],
         [ 0.5926, -7.3648,  4.3767],
         [ -5.9890,  0.3879,  3.8754],
         [ 3.0135, -6.8771, -7.4405]]
py_layer3 = hcat(py_layer3...)
py_layer3 = py_layer3'

py_layer4 = [ 1.5361, -1.5164, -1.5870,  1.8060]

4-element Vector{Float64}:
  1.5361
 -1.5164
 -1.587
  1.806

In [4]:
Flux.params(nn)[1] .= py_layer1
Flux.params(nn)[2] .= py_layer2
Flux.params(nn)[3] .= py_layer3
Flux.params(nn)[4] .= py_layer4

4-element Vector{Float32}:
  1.5361
 -1.5164
 -1.587
  1.806

In [ ]:
using LaplaceRedux
la = Laplace(nn; likelihood=:classification)
fit!(la, data)
optimize_prior!(la; verbose=true, n_steps=100)

[ Info: Precompiling LaplaceRedux [c52c1a26-f7c5-402b-80be-ba1e638ad478]


In [ ]:
using Plots
_labels = sort(unique(y))
plt_list = []
for target in _labels
    plt = plot(la, X, y; target=target, clim=(0,1))
    push!(plt_list, plt)
end
plot(plt_list...)

In [ ]:
predictions_probit = reduce(hcat, predict(la, X))
predictions_probit = reshape(predictions_probit, 4, 100)

In [ ]:
_labels = sort(unique(y))
plt_list = []
for target in _labels
    plt = plot(la, X, y; target=target, clim=(0,1), link_approx=:plugin)
    push!(plt_list, plt)
end
plot(plt_list...)

In [ ]:
predictions_plugin = reduce(hcat, predict(la, X, link_approx=:plugin))
predictions_plugin = reshape(predictions_plugin, 4, 100)

In [ ]:
@show la.H